### Practice 5:RNN 垃圾郵件分類 : spam.csv (spam: (電腦)垃圾郵件)

In [1]:
import numpy as np
import pandas as pd
import torch
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

df = pd.read_csv('./spam.csv')
display(df.head())
reviews = np.asarray(df['sms'])
labels =  np.asarray(df['label'])
print('reviews[:5]=\n',reviews[:5])
print('labels[:5]=\n',labels[:5])

Using cpu device


,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


reviews[:5]=
 ['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'
 'Ok lar... Joking wif u oni...'
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
 'U dun say so early hor... U c already then say...'
 "Nah I don't think he goes to usf, he lives around here though"]
labels[:5]=
 ['ham' 'ham' 'spam' 'ham' 'ham']


In [2]:
reviews = [x.lower() for x in reviews]  # reviews 全部小寫
print(reviews[:5])
print(labels[:5])

['go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there got amore wat...', 'ok lar... joking wif u oni...', "free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive entry question(std txt rate)t&c's apply 08452810075over18's", 'u dun say so early hor... u c already then say...', "nah i don't think he goes to usf, he lives around here though"]
['ham' 'ham' 'spam' 'ham' 'ham']


In [3]:
from string import punctuation
text_string = '\n'.join(reviews) # 所有文本連接在一起
temp_str = ''
for i in text_string:
    if i not in punctuation:  # 不是標點符號 temp_str一直累加後面的字
        temp_str = temp_str + i
text_string = temp_str
print(text_string[:300])

go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
ok lar joking wif u oni
free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s
u dun say so early hor 


In [4]:
vocab = list(set(text_string.replace('\n',' ').split(' ')))  
print(vocab[:100])
print(len(vocab))

['', 'overdose', 'kr', 'daplease', 'cr01327bt', 'nearly', 'hows', 'basic', 'discount', 'joinedso', '447801259231', 'handed', 'fireplace', 'hail', 'might', 'pdatenow', 'tarpon', 'irritating', 'polo', 'grooved', 'cal', 'out', 'baby', 'webpage', 'dime', 'vilikkamt', 'lancaster', 'activ8', 'stuff42moro', 'punish', 'burgundy', 'hear', 'fedex', 'deliveredtomorrow', 'sib', 'genius', 'convincingjust', 'audrey', 'stoptxtstopå£150week', 'opened', 'r', 'strangersaw', 'voucherstext', 'sary', 'cos', 'impatient', '8neighbour', '450p', 'aust', 'goodevening', '08717509990', 'intentions', '220cm2', 'asian', 'meummifyingbye', '020603', 'youåõre', 'attention', 'wikipediacom', 'footprints', 'splash', 'chip', 'ls15hb', 'dayhas', 'beloved', 'church', 'dave', 'sthis', 'labor', 'sitting', 'thesis', 'blessings', 'flood', 'arrange', 'financial', 'remet', 'property', 'networking', 'accenture', 'txtjourney', 'cl', 'specific', 'asia', 'prepared', 'upto', 'proper', 'profit', 'waitshould', 'real1', 'voicemail', 'sme

In [5]:
word2idx = {w: i for i, w in enumerate(vocab)}  
idx2word = {v: k for k, v in word2idx.items()}
print(word2idx)

{'': 0, 'overdose': 1, 'kr': 2, 'daplease': 3, 'cr01327bt': 4, 'nearly': 5, 'hows': 6, 'basic': 7, 'discount': 8, 'joinedso': 9, '447801259231': 10, 'handed': 11, 'fireplace': 12, 'hail': 13, 'might': 14, 'pdatenow': 15, 'tarpon': 16, 'irritating': 17, 'polo': 18, 'grooved': 19, 'cal': 20, 'out': 21, 'baby': 22, 'webpage': 23, 'dime': 24, 'vilikkamt': 25, 'lancaster': 26, 'activ8': 27, 'stuff42moro': 28, 'punish': 29, 'burgundy': 30, 'hear': 31, 'fedex': 32, 'deliveredtomorrow': 33, 'sib': 34, 'genius': 35, 'convincingjust': 36, 'audrey': 37, 'stoptxtstopå£150week': 38, 'opened': 39, 'r': 40, 'strangersaw': 41, 'voucherstext': 42, 'sary': 43, 'cos': 44, 'impatient': 45, '8neighbour': 46, '450p': 47, 'aust': 48, 'goodevening': 49, '08717509990': 50, 'intentions': 51, '220cm2': 52, 'asian': 53, 'meummifyingbye': 54, '020603': 55, 'youåõre': 56, 'attention': 57, 'wikipediacom': 58, 'footprints': 59, 'splash': 60, 'chip': 61, 'ls15hb': 62, 'dayhas': 63, 'beloved': 64, 'church': 65, 'dave':

In [6]:
sentences = text_string.split('\n')
print(sentences[:5])

['go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat', 'ok lar joking wif u oni', 'free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s', 'u dun say so early hor u c already then say', 'nah i dont think he goes to usf he lives around here though']


In [ ]:
sentences_ints = []
for index, i in enumerate(sentences):
    word_list = i.replace('\n','').split(' ')
    sentences_ints.append(list())
    for j in word_list:
        sentences_ints[index].append(word2idx[j])
print(sentences_ints[:5])

In [ ]:
pad_seq_length = 30
for index, i in enumerate(sentences_ints):
    if len(i) > pad_seq_length:
        sentences_ints[index] =  sentences_ints[index][:pad_seq_length]
    else:
        sentences_ints[index] = sentences_ints[index] + [0]*(pad_seq_length-len(i))
print(sentences_ints[:5])

In [ ]:
sentences_ints = np.asarray(sentences_ints)
labels = np.array([1 if label == 'ham' else 0 for label in labels])
print(sentences_ints[:5])
print(labels[:5])

In [ ]:
num_docs =  len(sentences_ints)
frac = 0.8
X_train = sentences_ints[0:int(frac*num_docs)]
X_test = sentences_ints[int(frac*num_docs):]
y_train = labels[0:int(frac*num_docs)]
y_test = labels[int(frac*num_docs):]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
batch_size = 64
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
import torch.nn as nn
sequence_length = pad_seq_length
input_size = 30
hidden_size = 128
num_layers = 2
num_classes = 2
num_epochs = 5
learning_rate = 0.01
embedding_dim = 30
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, vocab_size, embedding_dim):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        x = x.type(torch.LongTensor).to(device)
        x = self.embedding(x).to(device)
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out
model = RNN(input_size, hidden_size, num_layers, num_classes, len(vocab), embedding_dim).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (texts, labels) in enumerate(train_loader):
        texts = texts.to(device)
        labels = labels.type(torch.LongTensor).to(device)
        # Forward pass
        outputs = model(texts)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i) % 10 == 0:
            print('loss: {}'.format(loss.item()))
model.eval()
with torch.no_grad():
    correct = 0 ;  total = 0
    for texts, labels in test_loader:
        texts = texts.to(device)
        labels = labels.type(torch.LongTensor).to(device)
        outputs = model(texts)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the 10000 test images: ',(100 * correct / total)) 